In [ ]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, save_img
from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layersimport Dense, GlobalAveragePooling2D
from skimage.restoration import denoise_nl_means, estimate_sigma
from skimage.transform import resize
import zipfile
import shutil
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.preprocessing import LabelEncoder

In [ ]:
# Install and configure Kaggle
!pip install kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d kmader/skin-cancer-mnist-ham10000

# Extract the dataset
with zipfile.ZipFile('/content/skin-cancer-mnist-ham10000.zip', 'r') as zip_ref:
zip_ref.extractall('dataset')

# Load metadata
df = pd.read_csv("/content/dataset/HAM10000_metadata.csv")
print(df)
print(df['dx'].value_counts())
df.drop(['lesion_id', 'dx_type', 'sex', 'localization', 'age'], axis=1, inplace=True)
df = df.rename(columns={'dx': 'label'})
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])
df['label'] = df['label'].astype(str)
print(df['label'].value_counts())

# Copy images to a single directory
source_folder1 = '/content/dataset/HAM10000_images_part_1'